Embedding後的資料集已儲存至CSV檔，此步驟可省略

直接進行上傳即可

# Install depencies

In [6]:
!pip3 install regex pandas tiktoken pinecone-client openai tqdm numpy --quiet

# Download Dataset

In [9]:
import json
import re
import pandas
import wget

In [11]:
wget.download("https://gnehs.github.io/ntut-course-crawler-node/111/2/main.json",out = "datasets/111-2.json")
wget.download("https://gnehs.github.io/ntut-course-crawler-node/112/1/main.json",out = "datasets/112-1.json")

'datasets/112-1.json'

# Remove meaningless words

In [5]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    cleaned_text = re.sub(url_pattern, '', text)
    cleaned_text = cleaned_text.replace("，請參閱連結:", ' ')
    return cleaned_text

data = json.load(open("111-2.json"))
formated = []

formated = []
for course in data:
    temp=course["description"]["zh"]
    if(len(course["description"]["zh"])!=0 and not ("學院指定向度" in temp) and len(course["teacher"])!=0 and len(course["classroom"])!=0):
        formated.append([course["id"],remove_urls(course["name"]["zh"]),course["description"]["zh"]])
df=pandas.DataFrame(formated,columns=["id","title","description"])
df.head(5)

,id,title,description
0,310735,教育議題專題,本課程主要探討當前重要教育議題，如：藝術與美感教育、性別教育、人權教育、勞動教育、法治教育、...
1,310736,教育哲學,本課程旨在教導學生一個簡潔、廣泛又清楚的教育哲學架構，課程包 括三大部份，第一部份探討哲學的...
2,310737,學習評量,本課程係在介紹教育測驗及多元評量之原理原則及實務操作，讓學習者在未來從事教職工作能有效應用於...
3,310738,班級經營,本課程目標如下：(一) 充實學生各種激發快樂學習、健康成長的學識，以培養適任的優良教師；(二...
4,310739,土木與建築群教學實習,本課程的目標是教導有志從事土木與建築群教學工作的學生，使其能熟悉土木與建築群課程綱要，準備分...


In [6]:
df["id"]=pandas.to_numeric(df["id"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1582 entries, 0 to 1581
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           1582 non-null   int64 
 1   title        1582 non-null   object
 2   description  1582 non-null   object
dtypes: int64(1), object(2)
memory usage: 37.2+ KB


# Embedding Datasets

## Initialize embedding model

In [8]:
import tiktoken
from tqdm.notebook import tqdm
from openai.embeddings_utils import get_embedding
import openai
openai.api_key=""
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base" 
encoding = tiktoken.get_encoding(embedding_encoding)


In [18]:
test_df=df.copy()
progress_bar = tqdm(total=len(test_df))
def convert_embedding(temp,progress_bar):
    progress_bar.update(1)
    return get_embedding(temp, engine=embedding_model)
test_df["description_embedding"]=test_df["description"].copy().apply(lambda temp:convert_embedding(temp,progress_bar))

  0%|          | 0/1582 [00:00<?, ?it/s]

In [19]:
test_df

,id,title,description,description_embedding
0,310735,教育議題專題,本課程主要探討當前重要教育議題，如：藝術與美感教育、性別教育、人權教育、勞動教育、法治教育、...,"[0.010150860995054245, -0.008742461912333965, ..."
1,310736,教育哲學,本課程旨在教導學生一個簡潔、廣泛又清楚的教育哲學架構，課程包 括三大部份，第一部份探討哲學的...,"[0.018458586186170578, -0.003821807447820902, ..."
2,310737,學習評量,本課程係在介紹教育測驗及多元評量之原理原則及實務操作，讓學習者在未來從事教職工作能有效應用於...,"[-0.012573255226016045, 0.0014721325132995844,..."
3,310738,班級經營,本課程目標如下：(一) 充實學生各種激發快樂學習、健康成長的學識，以培養適任的優良教師；(二...,"[-0.007300455588847399, -0.0012471340596675873..."
4,310739,土木與建築群教學實習,本課程的目標是教導有志從事土木與建築群教學工作的學生，使其能熟悉土木與建築群課程綱要，準備分...,"[-0.0030072827357798815, -0.01690831035375595,..."
...,...,...,...,...
1577,318002,語文測驗 (四),本課主旨在設計課程，提供練習，俾使學生嫻熟各式的英語測驗，如全民英檢、托福測驗、TOEIC ...,"[-0.0002108228363795206, -0.006927439942955971..."
1578,318003,語文測驗 (四),本課主旨在設計課程，提供練習，俾使學生嫻熟各式的英語測驗，如全民英檢、托福測驗、TOEIC ...,"[-0.0007966117700561881, -0.008518671616911888..."
1579,318006,進階 C 語言實務,C 語言為開發作業系統、嵌入式系統、通訊協定等程式最常使用的程式語言，本課程主要教導 C 程...,"[0.011034101247787476, -0.014431698247790337, ..."
1580,318008,音樂欣賞,本課程以培養學生尊重與欣賞不同族群、不同時空、不同價值觀所產生的音樂而設。藉由對音樂元素、本...,"[0.009749340824782848, 0.000817790103610605, 0..."


## Save embedding datasets for reuse

In [132]:
test_df.to_csv("111-2embedding.csv")

# Upload to Pinecone Database

In [ ]:
import pandas as pd
import pinecone
from ast import literal_eval
from typing import List, Iterator
import numpy as np
import openai 
openai.api_key=""
EMBEDDING_MODEL = "text-embedding-ada-002"
df = pd.read_csv("./datasets/111-2embedding.csv")
df['description_embedding'] = df.description_embedding.apply(literal_eval)
df['id'] = df['id'].apply(str)
api_key=""
pinecone.init(api_key=api_key,environment="northamerica-northeast1-gcp")


# Models a simple batch generator that make chunks out of an input DataFrame
class BatchGenerator:
    
    
    def __init__(self, batch_size: int = 10) -> None:
        self.batch_size = batch_size
    
    # Makes chunks out of an input DataFrame
    def to_batches(self, df: pd.DataFrame) -> Iterator[pd.DataFrame]:
        splits = self.splits_num(df.shape[0])
        if splits <= 1:
            yield df
        else:
            for chunk in np.array_split(df, splits):
                yield chunk

    # Determines how many chunks DataFrame contains
    def splits_num(self, elements: int) -> int:
        return round(elements / self.batch_size)
    
    __call__ = to_batches

df_batcher = BatchGenerator(300)

# Pick a name for the new index
index_name = 'ntut-courses'

# Check whether the index with the same name already exists - if so, delete it
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)
    
# # Creates new index
pinecone.create_index(name=index_name, dimension=len(df['description_embedding'][0]))
index = pinecone.Index(index_name=index_name)

# Confirm our index was created
pinecone.list_indexes()

for batch_df in df_batcher(df):
    index.upsert(vectors=zip(batch_df.id, batch_df.description_embedding), namespace='content')

index.describe_index_stats()